# Motivation

The idea is to analyze the tags provided on various problems that feature in programming contests to quantitatively see
which type of problems are most common at various rating divisions.

## Codeforces

For Codeforces, the common rating divisions are Div. 3, Div 2. and Div 1. with the problems ordered from least to most
difficult. It would make sense to first identify the most recent contests of these types, find the problems they contain,
and the corresponding tags. Then, we can visualize the distribution of the tags and see which are the most frequent,
along with possible secondary statistics like which tags are the "toughest" i.e., correspond to most failed attmepts,
which tags are the easiest and so on.

Relevant APIs are -
* `https://codeforces.com/api/contest.list`
* `https://codeforces.com/api/problemset.problems`

The first returns a list of all contests as a [`Contest`](https://codeforces.com/apiHelp/objects#Contest) object.

```json
{
    "id": 1854,
    "name": "Codeforces Round 889 (Div. 1)",
    "type": "CF",
    "phase": "BEFORE",
    "frozen": false,
    "durationSeconds": 9000,
    "startTimeSeconds": 1690641300,
    "relativeTimeSeconds": -147153
}
```

Relevant fields are only `id` and `name`. We need string-based heuristics for filtering the name to find the standard
Codeforces rounds and their intended "Div"s.

The problemset API returns a list of all problems as a [`Problem`]() object.

```json
{
    "contestId": 1853,
    "index": "B",
    "name": "Fibonaccharsis",
    "type": "PROGRAMMING",
    "points": 1000.0,
    "rating": 1200,
    "tags": ["binary search", "brute force", "math"]
}
```

Relevant fields are `contestId`, to tie back to the contest and check whether it came from a regular CF Round, and which
Div contest it came from. `rating` is also a useful feature to denote difficulty, and may be used skip the contest check
entirely. `tags` is obviously the one we want.

### Get data

In [1]:
import requests
r = requests.get('https://codeforces.com/api/problemset.problems')
r.status_code

200

In [7]:
from typing import List

class Problem:
    def __init__(self, name: str, rating: int, tags: List[str]):
        self.name = name
        self.rating = rating
        self.tags = tags
    
    def __repr__(self):
        return f'Problem(name={str(self.name)}, rating={str(self.rating)}, tags={str(self.tags)})'
    
    def __str__(self):
        return repr(self)

In [12]:
data = r.json()
problems = data['result']['problems']
print(problems[0])

{'contestId': 1853, 'index': 'B', 'name': 'Fibonaccharsis', 'type': 'PROGRAMMING', 'points': 1000.0, 'rating': 1200, 'tags': ['binary search', 'brute force', 'math']}


In [13]:
import numpy as np
import pandas as pd

problems = [Problem(problem['name'], problem.get('rating', np.nan), problem.get('tags', [])) for problem in problems]
print(problems[0])

Problem(name=Fibonaccharsis, rating=1200, tags=['binary search', 'brute force', 'math'])


In [14]:
names = [problem.name for problem in problems]
ratings = [problem.rating for problem in problems]
tags = [problem.tags for problem in problems]

df = pd.DataFrame({
    'name': names,
    'rating': ratings,
    'tags': tags
})

print(df)

                        name  rating  \
0             Fibonaccharsis  1200.0   
1                  Desorting   800.0   
2              Panda Meetups  3500.0   
3                  Rivalries  3400.0   
4     Miriany and Matchstick  2800.0   
...                      ...     ...   
8800     The least round way  2000.0   
8801                  Winner  1500.0   
8802  Ancient Berland Circus  2100.0   
8803             Spreadsheet  1600.0   
8804          Theatre Square  1000.0   

                                                   tags  
0                    [binary search, brute force, math]  
1                           [brute force, greedy, math]  
2                          [data structures, dp, flows]  
3     [constructive algorithms, data structures, gre...  
4                 [constructive algorithms, dp, greedy]  
...                                                 ...  
8800                                         [dp, math]  
8801                          [hashing, implementation]

### Preprocessing

In [17]:
all_tags = set(tag for tags_list in df['tags'] for tag in tags_list)
print(len(all_tags))
print(all_tags)

37
{'hashing', 'string suffix structures', 'meet-in-the-middle', 'games', 'chinese remainder theorem', 'interactive', 'strings', 'binary search', 'bitmasks', 'expression parsing', 'fft', 'two pointers', 'ternary search', 'implementation', 'brute force', 'geometry', 'shortest paths', 'trees', 'dsu', 'graphs', '2-sat', 'schedules', 'graph matchings', 'flows', 'matrices', '*special', 'sortings', 'greedy', 'combinatorics', 'data structures', 'dfs and similar', 'dp', 'math', 'number theory', 'divide and conquer', 'constructive algorithms', 'probabilities'}


In [18]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
tags_encoded = pd.DataFrame(mlb.fit_transform(df['tags']), columns=mlb.classes_, index=df.index)
df = pd.concat([df, tags_encoded], axis=1)
print(df)

                        name  rating  \
0             Fibonaccharsis  1200.0   
1                  Desorting   800.0   
2              Panda Meetups  3500.0   
3                  Rivalries  3400.0   
4     Miriany and Matchstick  2800.0   
...                      ...     ...   
8800     The least round way  2000.0   
8801                  Winner  1500.0   
8802  Ancient Berland Circus  2100.0   
8803             Spreadsheet  1600.0   
8804          Theatre Square  1000.0   

                                                   tags  *special  2-sat  \
0                    [binary search, brute force, math]         0      0   
1                           [brute force, greedy, math]         0      0   
2                          [data structures, dp, flows]         0      0   
3     [constructive algorithms, data structures, gre...         0      0   
4                 [constructive algorithms, dp, greedy]         0      0   
...                                                 ...       .

### Visualization

In [26]:
from lets_plot import *
LetsPlot.setup_html()

ggplot(df, aes(x='shortest paths', fill='rating')) + ggsize(700, 300) + \
    geom_density(color='dark_green', alpha=.7) + scale_fill_brewer(type='seq') + \
    theme(panel_grid_major_x='blank') + flavor_solarized_light()

## CodeChef

For CodeChef, we have two rating divisions of Div 2., and Div 1. There are long contests, short contests and Snacktime.
For each of these, we can repeat the same process as in Codeforces.